---
toc: false
comments: true
layout: post
title: Deployment
description: Personalized Deployment
permalink: /csp/sprint5/deployment
courses: { csp: {week: 18} }
---

## Basic rundown
Deployment is making your website public to anyone on the internet without the need to install anything locally. However, in order for this to work we need make sure it works locally because if it doesn't work locally it will not work in production.

## Steps to deployment

### 1. Test locally
We need to test it locally before deployment. As I wrote before, if it doesn't work locally then it won't work in deployment.

### 2. Make sure all files are correct for our website
Nginx

In [ ]:
server {
    listen 80;
    listen [::]:80;
    server_name motor.stu.nighthawkcodingsociety.com ; # Change server name to the one on R53
    # Configure CORS Headers
    location / {
        proxy_pass http://localhost:8104; # Change port to port on docker
        # Simple requests
        if ($request_method ~* "(GET|POST|PUT|DELETE)") { # Customize Request methods based on your needs
                add_header "Access-Control-Allow-Origin"  ;
        }
        # Preflighted requests
        if ($request_method = OPTIONS ) {
                add_header "Access-Control-Allow-Origin"  ;
                add_header "Access-Control-Allow-Methods" "GET, POST, PUT, DELETE, OPTIONS, HEAD"; # Make sure the request methods above match here
                add_header "Access-Control-Allow-Headers" "Authorization, Origin, X-Requested-With, Content-Type, Accept";
                return 200;
        }
    }
}

In [ ]:
docker-compose

In [ ]:
version: '3'
services:
        web:
                image: motorsports
                build: .
                env_file:
                        - .env # This file is optional; defaults will be used if it does not exist
                ports:
                        - "8104:8104"
                volumes:
                        - ./instance:/instance
                restart: unless-stopped

In [ ]:
dockerfile

In [ ]:
FROM docker.io/python:3.12.8

WORKDIR /

# --- [Install python and pip] ---
RUN apt-get update && apt-get upgrade -y && \
    apt-get install -y python3 python3-pip git
COPY . /

RUN pip install --no-cache-dir -r requirements.txt
RUN pip install gunicorn
RUN ./scripts/db_init.py

ENV GUNICORN_CMD_ARGS="--workers=3 --bind=0.0.0.0:8104"

EXPOSE 8104

# Define environment variable
ENV FLASK_ENV=production

CMD [ "gunicorn", "main:app" ]

### 3. Using cockpit to deploy to AWS

In [ ]:
git clone https://github.com/Tvick22/personal_flocker_backend.git motor_backend
cd motor_backend

In [ ]:
touch .env

In [ ]:
ADMIN_USER='toby'
ADMIN_PASSWORD='123Toby!'
DEFAULT_USER='hop'
DEFAULT_PASSWORD='123Hop!'

In [ ]:
RUN ./scripts/db_init.py

In [ ]:
docker-compose build

In [ ]:
docker-compose up -d

In [ ]:
docker ps

In [ ]:
curl localhost:8104

### 4. Configure DNS
Using route 53 the record can be created with the appropriate values

## Layers

![stack]({{site.baseurl}}/images/stack.png)